## Single linked list

Trong trường hợp Stack được cài đặt bằng linked list thì chi phí của tất cả các hàm <br>
pop(), push(), top(), len(), is_empty() đều là O(1) khác với khi cài đặt bằng mảng Array, vì ở đây không có phi phí resize kích thước mảng (amortized cost).

In [3]:
class Empty(Exception):
    pass



In [4]:
class LinkedStack:
    class _Node:
        __slots__ = '_element', '_next'
        
        def __init__(self, element, next):
            self._element = element
            self._next = next;
        
    def __init__(self):
        self._head = None
        self._size = 0
    
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def push(self, e):
        self._head = self._Node(e, self._head)
        self._size = 1
        
    def top(self):
        if self.is_empty():
            raise Empty('Stack is empty')
            
        return self._head._element
    
    def pop(self):
        if self.is_empty():
            raise Empty('Stack is empty')
        
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        return answer
    

In [6]:
class LinkedQueue:
    class _Node:
        __slots__ =  '_element', '_next'
        
        def __init__(self, element, next):
            self._element = element
            self._next = next
            
    def __init__(self):
        self._head = None
        self._tail = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('Queue is empty')
        
        answer = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
            
        return answer
    
    def enqueue(self, e):
        newest = self._Node(e, None)
        
        if self.is_empty():
            self._head = newest
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1
        
S = LinkedQueue()
S.enqueue(5)
S.enqueue(6)
S.enqueue(7)

print(S.dequeue())
print(S.dequeue())
print(S.dequeue())

5
6
7


## Circularly linked list

In [ ]:
class CircleQueue:
    class _Node:
         __slots__ =  '_element', '_next'
        
        def __init__(self, element, next):
            self._element = element
            self._next = next
            
    def __init__(self):
        self._tail = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def first(self):
        if self.is_empty():
            raise Empty('CircleQueue is empty')
            
        head = self._tail._next
        return head._element
    
    def dequeue(self):
        if self.is_empty():
            raise Empty('CircleQueue is empty')
        oldhead = self._tail._next
        if self._size == 1:
            self._tail = None
        else:
            self._tail._next = oldhead._next
        self._size -= 1
        return oldhead._element
        
    def enqueue(self, e):
        newest = self._Node(e, None)
        
        if self.is_empty():
            newest._next = newest
        else:
            newest._next = self._tail._next
            self._tail._next = newest
        self._tail = newest
        self._size += 1
    

## Double linked list

In [ ]:
class _DoubleLinkedBase:
    class _Node:
        __slots__ = '_element', "_next", "_prev"
        
        def __init__(self, e, prev, next):
            self._element = e
            self._prev = prev
            self._next = next
            
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._None(None, None, None)
        self._header._next = self._trailer   # trailer is after header
        self._trailer._prev = self._header   # header is before trailer
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def _insert_between(self, e, predecessor, successor):
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest
    
    def _delete_node(self, node):
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        element = node._element
        node._prev = node._next = node._element = None
        return element


In [ ]:
class LinkedDeque(_DoubleLinkedBase):
    def first(self):
        if self.is_empty():
            raise Empty('Deque is empty')
            
        return self._header._next._element
    
    def last(self):
        if self.is_empty():
            raise Empty('Deque is empty')
            
        return self._trailer._prev._element
    
    
    def insert_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def insert_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer._next)
    
    def delete_first(self):
        if self.is_empty():
            raise Empty('Deque is empty')
            
        return self._delete_node(self._head._next)
    
    def delete_last(self):
        if self.is_empty():
            raise Empty('Deque is empty')
        return self._delete_node(self._tail._prev)
    

In [ ]:
class PositionalList(_DoubleLinkedBase):
    
    #--------------- Nested Position class ----------------#
    class Position:
        def __init__(self, container, node):
            self._container = container
            self._node = node
            
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node
        
        def __ne__(self, other):
            return not (self == other)
    
    #-------------- Utility method ----------------------#
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise TypeError('p does not belong to this container')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node):
        if node is self._header or node is self._trailer:
            return None
        else:
            return self.Position(self, node)
        
    # ------------ Accessor method ---------------------#
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
            
    #------------ Mutator ----------------------------#
    def _insert_between(self, e, predecessor, successor):
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original._prev, orignal)
    
    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, orginal._next)
    
    def delete(self, p):
        orginal = self._validate(p)
        return self._delele_node(original)
    
    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value
        